<a href="https://colab.research.google.com/github/romanzelararg/Tecnicas-de-Procesamiento-del-Habla-Grupo11/blob/main/Modelo2_act_Proc_Habla_RN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO DE RED NEURONAL CON CAPAS DENSAS.

#**Procesamiento de Texto**






#Preparación del área de trabajo

Importar Librerias

In [ ]:
!pip install nltk
!pip install unidecode
!pip install vaderSentiment

In [ ]:
import os  # Se agrega esta importación para manejar archivos
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import MaxAbsScaler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud  # Se agrega esta importación para la nube de palabras


In [ ]:
# Descargar recursos de NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Cargar Documento

In [ ]:
# Verificar si la ruta existe
if os.path.exists(libro):
    print("El archivo existe y la ruta es correcta.")
else:
    print("El archivo no existe o la ruta es incorrecta.")

# Leer y limpiar el texto
with open(libro, 'r') as f:
    texto = f.read()

El archivo existe y la ruta es correcta.


# Preprocesamiento y Análisis de Sentimiento

#Limpieza y Tokenización de texto

In [ ]:
# Función de limpieza de texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'\W', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)
    texto = re.sub(r'\@\w+|\#', '', texto)
    return texto

In [ ]:
# Función para lematizar texto
def lematizar_texto(tokens):
    lemmatizer = WordNetLemmatizer()
    tokens_lemmatizados = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens_lemmatizados

In [ ]:
texto_limpio = limpiar_texto(texto)

In [ ]:
# Tokenización
tokens = nltk.word_tokenize(texto_limpio)

print('Cantidad de tokens unicos',len(tokens))

Cantidad de tokens unicos 22637


In [ ]:
#Fraccion de Stopwords en un corpus

def stopwords_percentage(text):
  '''
  aqui usamos un recurso léxico (stopwords) para filtrar un corpus
  '''
  stopwd = stopwords.words('spanish')
  content = [w  for w in text if w.lower() not in stopwd]
  return len(content)/len(text)

stopwords_percentage(tokens)

0.5240535406635155

In [ ]:
#Limpieza de texto

# Eliminar stopwords
stop_words = set(stopwords.words('spanish'))
tokens_filtrados = [token for token in tokens if token not in stop_words]

print('Cantidad de tokens sin stopwords',len(tokens_filtrados))

Cantidad de tokens sin stopwords 11863


In [ ]:
# Lematización
tokens_lemmatizados = lematizar_texto(tokens_filtrados)

In [ ]:
# Inicializar el analizador de sentimiento VADER
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Analizar sentimiento de los tokens
def analizar_sentimiento(tokens):
    etiquetas_sentimiento = []
    for token in tokens:
        score = analyzer.polarity_scores(token)
        if score['compound'] >= 0.05:
            etiquetas_sentimiento.append(1)  # Positivo
        elif score['compound'] <= -0.05:
            etiquetas_sentimiento.append(-1)  # Negativo
        else:
            etiquetas_sentimiento.append(0)  # Neutral
    return etiquetas_sentimiento


In [ ]:
# Obtener etiquetas de sentimiento para los tokens
etiquetas_sentimiento = analizar_sentimiento(tokens_lemmatizados)

# Convertir a numpy array
y = np.array(etiquetas_sentimiento)

print('Tamaño de y:', y.shape)

Tamaño de y: (11863,)


# Construcción del Modelo de Red Neuronal

In [ ]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tokens_lemmatizados)

print('Tamaño de la matriz TF-IDF:', X.shape)

Tamaño de la matriz TF-IDF: (11863, 4920)


# Asegurar que las longitudes de X e y son consistentes



In [ ]:
if X.shape[0] != len(y):
    raise ValueError(f"Dimensiones inconsistentes: X tiene {X.shape[0]} muestras, pero y tiene {len(y)} muestras.")

#Construcción del Modelo de Red Neuronal

In [ ]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Balancear clases usando SMOTE
smote = SMOTE()
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

Dividir los datos en conjuntos de entrenamiento y prueba

Normalizacion

In [ ]:
# Normalización de los datos de entrada
scaler = MaxAbsScaler()
X_train_normalized = scaler.fit_transform(X_train_resampled.toarray())
X_test_normalized = scaler.transform(X_test.toarray())

Aquitectura del modelo

In [ ]:
# Definir la arquitectura del modelo
model = Sequential([
    Dense(128, input_shape=(X_train_normalized.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Resumen del modelo
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 128)               629888    
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 638209 (2.43 MB)
Trainable params: 638209 (2.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#Entrenamiento y Evaluación

Entranamiento

In [ ]:
# Entrenar el modelo
history = model.fit(X_train_normalized, y_train_resampled, epochs=50, batch_size=128, validation_split=0.2)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_normalized, y_test)
print('Accuracy:', accuracy)

Epoch 1/50
177/177 [==============================] - 6s 30ms/step - loss: -13.6705 - accuracy: 0.4300 - val_loss: 6.9082 - val_accuracy: 0.4227
Epoch 2/50
177/177 [==============================] - 4s 20ms/step - loss: -301.5914 - accuracy: 0.4919 - val_loss: 44.1799 - val_accuracy: 0.5243
Epoch 3/50
177/177 [==============================] - 3s 19ms/step - loss: -1467.9038 - accuracy: 0.4999 - val_loss: 138.8980 - val_accuracy: 0.5551
Epoch 4/50
177/177 [==============================] - 3s 18ms/step - loss: -4075.1003 - accuracy: 0.5013 - val_loss: 313.4719 - val_accuracy: 0.5699
Epoch 5/50
177/177 [==============================] - 4s 24ms/step - loss: -8547.4893 - accuracy: 0.5031 - val_loss: 604.5681 - val_accuracy: 0.5683
Epoch 6/50
177/177 [==============================] - 4s 21ms/step - loss: -15272.6182 - accuracy: 0.5043 - val_loss: 1038.6909 - val_accuracy: 0.5611
Epoch 7/50
177/177 [==============================] - 3s 18ms/step - loss: -24546.1895 - accuracy: 0.5026 - va

Matiz de Confusion

In [ ]:
# Realizar predicciones
y_pred = model.predict(X_test_normalized)

# Convertir las predicciones a etiquetas binarias
y_pred_binary = (y_pred > 0.5).astype(int)

# Mostrar reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred_binary, target_names = ['Negativo', 'Positivo', 'Neutral']))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_binary)
print("Matriz de Confusión:")
print(conf_matrix)

75/75 [==============================] - 0s 3ms/step
Reporte de Clasificación:
              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00         9
    Positivo       0.99      1.00      1.00      2351
     Neutral       1.00      0.38      0.56        13

    accuracy                           0.99      2373
   macro avg       0.66      0.46      0.52      2373
weighted avg       0.99      0.99      0.99      2373

Matriz de Confusión:
[[   0    9    0]
 [   0 2351    0]
 [   0    8    5]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
